In [1]:
import json
import os
import pandas as pd
import shutil
from seeq import spy

# Set the compatibility option so that you maximize the chance that SPy will remain compatible with your notebook/script
spy.options.compatibility = 199

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# Workbook Jobs

In [spy.workbooks.ipynb](spy.workbooks.ipynb), you can learn to push and pull workbooks (Workbench Analyses and Organizer Topics) to/from the Seeq service/server using SPy.

You may need to do something "in bulk," in one of the following scenarios:

- Re-mapping references (e.g. historian tags/signals) from one datasource to another, or one asset tree to another
- Transferring work from one Seeq service/server to another, possibly including data

The set of functions in the `spy.workbooks.job` module are suitable for this work. Each function operates within a "job folder" that captures the state of the job. Unlike `spy.workbooks.pull()` and `spy.workbooks.push()`, the equivalent commands in `spy.workbooks.job` do not require all workbooks to be held in memory. This allows very large jobs to be executed (as long as there is sufficient disk space). All parts of the process are _resumable_, and SPy will pick up where it left off if the operation is interrupted for any reason (e.g. a network error).

This notebook will walk through the use of this module, referencing the scenarios above. In general, commands are executed in the following order:

1. `spy.workbooks.job.pull()`
2. `spy.workbooks.job.data.pull()` (optional)
3. `spy.workbooks.job.push()`
4. `spy.workbooks.job.data.push()` (optional)

## Establish the Job Folder

The parameter that defines a job is a _job folder_. It is the first argument for all job functions, and it is managed entirely by SPy. The folder is laid out in an intuitive way that allows you to inspect it, and, in some troubleshooting cases, make modifications yourself.

In [3]:
job_folder = 'Output/My First Workbooks Job'

# Remove the job folder so that old file/artifacts don't affect the tutorial
if os.path.exists(job_folder):
    shutil.rmtree(job_folder)

## Let's Make Something to Work With...

We need some Analyses/Topics to work with for the purposes of demonstrating the functionality, so let's make sure the example workbooks have been pushed.

In [4]:
example_workbooks = spy.workbooks.load('Support Files/Example Export.zip')
spy.workbooks.push(example_workbooks,
                   path='SPy Documentation Examples >> Workbook Job Import',
                   label=f'{spy.session.user.name} Workbook Job Example', 
                   refresh=False, 
                   errors='raise')

ID              Name      Type  \
0  D833DC83-9A38-48DE-BF45-EB787E9E8375  Example Analysis  Workbook   
1  811B1488-297A-4FD2-AE7C-A1FE0E3B3641     Example Topic  Workbook   

  Workbook Type Count            Time Errors   Result  \
0      Analysis    66  0:00:02.249930      0  Success   
1         Topic     5  0:00:00.766855      0  Success   

                     Pushed Workbook ID  \
0  0F0E5195-1D10-6000-960F-3D54376DAB21   
1  0F0E5195-69F5-60F0-BD79-CE8CFBA38F4E   

                                                 URL  
0  http://localhost:34216/0F0E5195-1C34-6460-88E5...  
1  http://localhost:34216/0F0E5195-1C34-6460-88E5...

## Pulling Workbooks

Start the job cycle by issuing the `spy.workbooks.job.pull()` to grab a set of workbooks and write them to disk.

As with `spy.workbooks.pull()`, we create a DataFrame full of workbooks to pull by using the `spy.workbooks.search()` function. Then we can supply that DataFrame to `spy.workbooks.job.pull()`, which takes many of the same parameters as `spy.workbooks.pull()`.

In [5]:
workbooks_df = spy.workbooks.search({
    'Path': 'SPy Documentation Examples >> Workbook Job Import'
})

# Store these in variables that we'll use later
example_analysis_workbook_id = workbooks_df[workbooks_df['Name'] == 'Example Analysis'].iloc[0]['ID']
example_topic_workbook_id = workbooks_df[workbooks_df['Name'] == 'Example Topic'].iloc[0]['ID']

workbooks_df

ID      Type Workbook Type  \
0  0F0E5195-1D10-6000-960F-3D54376DAB21  Workbook      Analysis   
1  0F0E5195-69F5-60F0-BD79-CE8CFBA38F4E  Workbook         Topic   

                                                Path              Name  \
0  SPy Documentation Examples >> Workbook Job Import  Example Analysis   
1  SPy Documentation Examples >> Workbook Job Import     Example Topic   

   Archived  Pinned                          Created At  \
0       NaN     NaN 2025-12-30 00:48:55.810988100+00:00   
1       NaN     NaN 2025-12-30 00:49:03.871193100+00:00   

                           Updated At     Owner Name Owner Username  \
0 2025-12-30 01:21:11.452407800+00:00  Agent API Key  agent_api_key   
1 2025-12-30 00:49:03.871193100+00:00  Agent API Key  agent_api_key   

                               Owner ID   Creator Name Creator Username  \
0  0F0E5193-930D-64D0-B261-8A6944987A34  Agent API Key    agent_api_key   
1  0F0E5193-930D-64D0-B261-8A6944987A34  Agent API Key    agent_api_key   

                             Creator ID                      Search Folder ID  
0  0F0E5193-930D-64D0-B261-8A6944987A34  0F0E5195-1C34-6460-88E5-A32177C3BD95  
1  0F0E5193-930D-64D0-B261-8A6944987A34  0F0E5195-1C34-6460-88E5-A32177C3BD95

In [6]:
spy.workbooks.job.pull(job_folder, workbooks_df)

ID  \
0  0F0E5195-1D10-6000-960F-3D54376DAB21   
1  0F0E5195-69F5-60F0-BD79-CE8CFBA38F4E   

                                                Path              Name  \
0  SPy Documentation Examples >> Workbook Job Import  Example Analysis   
1  SPy Documentation Examples >> Workbook Job Import     Example Topic   

  Workbook Type  Count                   Time  Errors   Result  
0      Analysis      0 0 days 00:00:00.778891       0  Success  
1         Topic      0 0 days 00:00:00.309327       0  Success

As mentioned earlier, jobs are _resumable_. If you execute the above cell again, you will see that the **Result** column indicates `Already pulled`.

If you would like to force a job to redo its work, supply the `resume=False` argument. You can also inspect the job folder's `Workbooks` subfolder and selectively delete workbook folders therein to force SPy to re-pull workbooks.

## Pushing Workbooks

As mentioned above, there are two primary scenarios where you want to push workbooks in bulk:

- Re-mapping references (e.g. historian tags/signals) from one datasource to another, or one asset tree to another
- Transferring work from one Seeq service/server to another, possibly including data

### Datasource Maps

In either case, it's important to understand the concept of _datasource maps_. These are JSON files that contain instructions for SPy as it maps the identifiers in the pulled workbook definitions to identifiers on the target system. These maps can incorporate relatively complex Regular Expression specifications that allow you to re-orient workbooks from one set of input data to another.

The `spy.workbooks.job.pull()` command will create a `Datasource Maps` folder inside the job folder. There will be one file for every datasource that was encountered during the pull operation -- if a workbook touched a datasource in some way, there will be a file for it.

Here's what a typical file looks like:

```
{
    "Datasource Class": "Time Series CSV Files",
    "Datasource ID": "Example Data",
    "Datasource Name": "Example Data",
    "Item-Level Map Files": [],
    "RegEx-Based Maps": [
        {
            "Old": {
                "Type": "(?<type>.*)",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Data ID": "(?<data_id>.*)"
            },
            "New": {
                "Type": "${type}",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Data ID": "${data_id}"
            }
        }
    ]
}
```

You can make modifications to these files by loading them into an editor, including Jupyter's text editor. Generally the most common action is to add or change entries in the `RegEx-Based Maps` block. That section is a _list_ of _dictionaries_ that each have an `Old` and a `New` subsection. Within the `Old` block, you can specify properties to match on. The key is the property name and the value is a [regular expression](https://en.wikipedia.org/wiki/Regular_expression), often employing a _capture group_. In the example above, the `Data ID` field is matching using the `.*` regex and storing it in a capture group called `data_id`. The `New` block then contains the properties and values to search upon to "map" to target items. In the example above, the `"Data ID": "${data_id}"` specification just means that the Data ID is being used "as-is" without any alteration.

(If you happen to be familiar with [Connector Property Transforms](https://telemetry.seeq.com/support-link/kb/latest/cloud/connector-property-transforms), this regex approach may feel familiar.)

Let's look at a more complicated example:

```
{
    "Datasource Class": "Time Series CSV Files",
    "Datasource ID": "Example Data",
    "Datasource Name": "Example Data",
    "Item-Level Map Files": [],
    "RegEx-Based Maps": [
        {
            "Old": {
                "Type": "(?<type>.*)",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Data ID": "(?<data_id>.*)"
            },
            "New": {
                "Type": "${type}",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Data ID": "${data_id}"
            }
        },
        {
            "Old": {
                "Type": "(?<type>.*)",
                "Path": "Example >> Cooling Tower 1",
                "Asset": "Area (?<subarea>[ABC])",
                "Name": "(?<name>.*)"
            },
            "New": {
                "Type": "${type}",
                "Path": "Example >> Cooling Tower 2",
                "Asset": "Area ${subarea}",
                "Name": "${name}"
            }
        }
    ]
}
```

In this example there are two RegEx-Based Maps specified. The first map is identical to the previous example, and it will be used first-- if there is not a match on the `Old` regex specifications, then SPy will move on to the next. The next map matches on a particular asset path (`Example >> Cooling Tower 1`) and a set of subareas (`A`, `B`, or `C`) and then maps them to the same area underneath `Example >> Cooling Tower 2`.

In this manner, you can use arbitrarily-complex mapping logic to accomplish the goal of re-mapping a workbook within the same Seeq server or properly mapping from one Seeq server to another.

#### Item-Level Map Files

As an alternative, or in addition, to `RegEx-Based Maps`, you can populate the `Item-Level Map Files` JSON array with a list of CSV files wherein each row directly maps from one item to another by `ID`. Here's what the CSV file might look like:

```
"Old ID","Old Name","New ID","New Name"
"62AD45C4-DEA1-477C-9C96-1D02F4EBA186","Temperature A1","61DA0A51-BD1C-4F56-943C-B908428DA847","Temperature P(A).PV"
"CA11EA1A-0826-4B33-BD4F-509381432E7D","Humidity A5","EE845784-C81D-4F4C-8A9F-A942377075F0","Humidity P(E).PV"
```

The important columns are `Old ID` and `New ID` -- all other columns are ignored.

You will generally only use this option if it's too difficult or impossible to describe the map using RegEx. Matches made via Item-Level Map Files take precedence over matches made by RegEx-Based Maps.

#### Datasource Mapping in Action

Let's run through an actual mapping scenario to see how it works and how to troubleshoot it when it goes wrong.

First we have to grab a couple of signal IDs so that we can use them later to illustrate some functionality.

In [7]:
area_a_temperature_id = spy.search({'Datasource Name': 'Example Data', 'Name': 'Area A_Temperature'}).iloc[0]['ID']
area_a_optimizer_id = spy.search({'Datasource Name': 'Example Data', 'Name': 'Area A_Optimizer'}).iloc[0]['ID']
area_a_wet_bulb_id = spy.search({'Datasource Name': 'Example Data', 'Name': 'Area A_Wet Bulb'}).iloc[0]['ID']
area_a_compressor_stage_id = spy.search({'Datasource Name': 'Example Data', 'Name': 'Area A_Compressor Stage'}).iloc[0]['ID']

First we will write out a CSV file to illustrate how the Item-Level Map Files work.

In [8]:
csv_df = pd.DataFrame([{
    'Old ID': area_a_temperature_id,
    'New ID': area_a_wet_bulb_id
}])
csv_df.to_csv('Output/item_level_map.csv')

Now we will write out a datasource map file that has, as its first map, a `New` block that will map to a `Name` that does not exist. This will let us see what happens both when the mapping is successful and when there are errors.

In [9]:
datasource_map = {
    "Datasource Class": "Time Series CSV Files",
    "Datasource ID": "Example Data",
    "Datasource Name": "Example Data",
    "Item-Level Map Files": [
        'Output/item_level_map.csv'
    ],
    "RegEx-Based Maps": [
        {
            "Old": {
                "Type": "(?<type>.*)",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Name": "Area A_Optimizer"
            },
            "New": {
                "Type": "${type}",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Name": "Area NonExistent_Optimizer"
            },
            # In this contrived example, if we match on the "Old" criteria, we don't want to continue to the next regex map
            "On Match": "Stop"
        },
        {
            "Old": {
                "Type": "(?<type>.*)",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Data ID": "(?<data_id>.*)"
            },
            "New": {
                "Type": "${type}",
                "Datasource Class": "Time Series CSV Files",
                "Datasource Name": "Example Data",
                "Data ID": "${data_id}"
            }        
        }
    ]
}

with open(os.path.join(job_folder, 'Datasource Maps', 'Datasource_Map_Time Series CSV Files_Example Data_Example Data.json'), 'w') as f:
    json.dump(datasource_map, f)

### Executing the Push

Now we push to server using a label that is guaranteed to differentiate our activity from other users. As you will see, there will be errors reported in the `Results` column because one item won't be mapped.

Note: You can supply `dry_run=True` to see a detailed log of what the push operation _would_ do if it actually wrote anything to the server. This can help you find errors in advance of actually making any changes in Seeq.

In [10]:
push_df = spy.workbooks.job.push(job_folder,
                                 path='SPy Documentation Examples >> Workbook Jobs',
                                 label=f'{spy.session.user.name} Workbook Job Example',
                                 errors='catalog')

You can see the error, but it's not formatted very well. So let's use a troubleshooting tool-- the "explain" function on the returned DataFrame:

In [11]:
print(push_df.spy.item_map.explain(area_a_optimizer_id))

StoredSignal "Area A_Optimizer" (0F0E5194-F120-7710-A8EE-4C2619321D62) not mapped, only override maps used
Using overrides from \\?\C:\GitHub\crab\sdk\pypi\seeq\spy\docs\Documentation\Output\My First Workbooks Job\Datasource Maps:
- Used "\\?\C:\GitHub\crab\sdk\pypi\seeq\spy\docs\Documentation\Output\My First Workbooks Job\Datasource Maps\Datasource_Map_Time Series CSV Files_Example Data_Example Data.json"
- Item-Level Map File "Output/item_level_map.csv" did not match for "Old ID" value "0F0E5194-F120-7710-A8EE-4C2619321D62"
- RegEx-Based Map 0: Item not found on server. Details:
    "Type"
        regex          "(?<type>.*)"
        matched on     "StoredSignal"
        searched for   "Signal"
    "Datasource Class"
        regex          "Time Series CSV Files"
        matched on     "Time Series CSV Files"
        searched for   "Time Series CSV Files"
    "Name"
        regex          "Area A_Optimizer"
        matched on     "Area A_Optimizer"
        searched for   "Area NonExi

This detailed explanation is intended to give you a starting point for troubleshooting. You can see the **regex** that was specified, the property values that were **matched on**, the **Capture groups** that resulted from the RegEx specifications, and the property values that were subsequently **searched for**. Since `Area NonExistent_Optimizer` does not exist, the explanation for `RegEx-Based Map 0` says _Item not found on server_.

Now let's look at the explanation for the successful maps (`Area A_Temperature`, `Area A_Compressor Stage`):

In [12]:
print(push_df.spy.item_map.explain(area_a_temperature_id))

Using overrides from \\?\C:\GitHub\crab\sdk\pypi\seeq\spy\docs\Documentation\Output\My First Workbooks Job\Datasource Maps:
- Used "\\?\C:\GitHub\crab\sdk\pypi\seeq\spy\docs\Documentation\Output\My First Workbooks Job\Datasource Maps\Datasource_Map_Time Series CSV Files_Example Data_Example Data.json"
Successful mapping:
  Old: StoredSignal "Area A_Temperature" (0F0E5194-F0F4-77F0-9282-75B6D1EDF8D0)
  New: StoredSignal "Area A_Wet Bulb" (0F0E5194-F638-6270-B8CD-969173BDFC9F)


In [13]:
print(push_df.spy.item_map.explain(area_a_compressor_stage_id))

Using overrides from \\?\C:\GitHub\crab\sdk\pypi\seeq\spy\docs\Documentation\Output\My First Workbooks Job\Datasource Maps:
- Used "\\?\C:\GitHub\crab\sdk\pypi\seeq\spy\docs\Documentation\Output\My First Workbooks Job\Datasource Maps\Datasource_Map_Time Series CSV Files_Example Data_Example Data.json"
- Item-Level Map File "Output/item_level_map.csv" did not match for "Old ID" value "0F0E5194-F480-FB30-9424-1E25FAFC6D1D"
- RegEx-Based Map 0: Unsuccessful match. Details:
    "Name"
        regex          "Area A_Optimizer"
        does not match "Area A_Compressor Stage"
- RegEx-Based Map 1: Successfully mapped. Details:
    "Type"
        regex          "(?<type>.*)"
        matched on     "StoredSignal"
        searched for   "Signal"
        and found      "StoredSignal"
    "Datasource Class"
        regex          "Time Series CSV Files"
        matched on     "Time Series CSV Files"
        searched for   "Time Series CSV Files"
        and found      "Time Series CSV Files"
    "

### Dummy Items

In cases where we couldn't map successfully, we can tell SPy to create "dummy" items. A dummy item is a signal, condition or scalar that has all the properties of the original item but has no data. (We'll show how to push data to dummy items later...)

Note the use of `create_dummy_items=True`, and also note `resume=False` so that SPy tries to push the workbooks again:

In [14]:
push_df = spy.workbooks.job.push(
    job_folder,
    path='SPy Documentation Examples >> Workbook Jobs',
    label=f'{spy.session.user.name} Workbook Job Example',
    create_dummy_items=True,
    resume=False,
    errors='catalog')

Now there are no errors, because any item that couldn't be mapped would be replaced by a dummy item.

Find the `Example Analysis` row of the output table above and click on the _link_ in the `URL` column to take a look at the resulting. You'll see that the **Details Pane** worksheet contains **Area A_Optimizer**, which is a blank "dummy item". 

We can look at what dummy items were created by inspecting `push_df.spy.item_map.dummy_items`. You can see that the `Name` is the same as the original and important properties like `Maximum Interpolation` have made their way to the dummy item.

In [15]:
push_df.spy.item_map.dummy_items

ID  \
0F0E5194-F120-7710-A8EE-4C2619321D62  0F0E51DD-89E7-7140-8E0B-D830C8445157   

                                     Archived Cache Enabled  \
0F0E5194-F120-7710-A8EE-4C2619321D62    False          True   

                                                                  Cache ID  \
0F0E5194-F120-7710-A8EE-4C2619321D62  0F0E5194-F120-7710-8B22-8DAADC3185C9   

                                                                                Data ID  \
0F0E5194-F120-7710-A8EE-4C2619321D62  [Agent API Key Workbook Job Example] 0F0E5194-...   

                                     Datasource Class  Datasource ID Enabled  \
0F0E5194-F120-7710-A8EE-4C2619321D62    Seeq Data Lab  Seeq Data Lab    True   

                                     Interpolation Method Locked  ...  \
0F0E5194-F120-7710-A8EE-4C2619321D62               Linear  False  ...   

                                     Original Datasource ID  \
0F0E5194-F120-7710-A8EE-4C2619321D62           Example Data   

                                                       Original Data ID  \
0F0E5194-F120-7710-A8EE-4C2619321D62  [Tag] Area A_Optimizer.sim.ts.csv   

                                                                 Scoped To  \
0F0E5194-F120-7710-A8EE-4C2619321D62  0F0E51DD-7995-FD50-961C-FBCC028D54F3   

                                     ExpectedProvidedType Formula  \
0F0E5194-F120-7710-A8EE-4C2619321D62                  NaN     NaN   

                                     Formula Parameters Formula Version  \
0F0E5194-F120-7710-A8EE-4C2619321D62                 []             NaN   

                                     UIConfig Custody ID Push Result  
0F0E5194-F120-7710-A8EE-4C2619321D62      NaN        NaN     Success  

[1 rows x 34 columns]

## Including Data

Dummy items are helpful, but they are "blank," they do not have any data associated with them. If you are transferring workbooks between servers and the destination server doesn't have access to the same datasources, it is useful to be able transfer the data itself from the source server to the dummy items on the destination server. A set of SPy functions is provided in `spy.workbooks.job.data` for this purpose.

As `spy.workbooks.job.pull()` pulls workbook information, it also tracks the usage of data items on Workbench Worksheets and in Organizer Topic Documents. This information is collated and saved to disk as the _data manifest_. You can inspect the manifest like so:

In [16]:
manifest_df = spy.workbooks.job.data.manifest(job_folder)

# Simplify the DataFrame so that it fits on the screen better
manifest_df[['ID', 'Path', 'Asset', 'Name', 'Start', 'End', 'Calculation']]

ID                        Path  \
0   0F0E5194-F0F4-77F0-9282-75B6D1EDF8D0                         NaN   
1   0F0E5194-F480-FB30-9424-1E25FAFC6D1D                         NaN   
2   0F0E5194-F13D-FBD0-A828-C62D76DBB3C7                         NaN   
3   0F0E5194-F120-7710-A8EE-4C2619321D62                         NaN   
4   0F0E5194-F638-6270-B8CD-969173BDFC9F                         NaN   
5   0F0E5194-F646-ECD0-B161-3B58818B56EA                         NaN   
6   0F0E5194-F5EC-7780-835D-E33F795E19DB                         NaN   
7   0F0E5194-F02C-64D0-B9EC-35732C8E310E  Example >> Cooling Tower 1   
8   0F0E5194-F58D-6410-8DD8-F446DC388736       Example >> Batch Data   
9   0F0E5194-F5D4-60E0-A95A-86D0659E009B       Example >> Batch Data   
10  0F0E5194-F0D2-7510-8A50-ECD4A148A666       Example >> Batch Data   

        Asset                      Name                            Start  \
0         NaN        Area A_Temperature 2018-11-01 12:36:56.759000+00:00   
1         NaN   Area A_Compressor Stage 2018-11-01 12:36:56.759000+00:00   
2         NaN  Area A_Relative Humidity 2018-11-01 12:36:56.759000+00:00   
3         NaN          Area A_Optimizer 2018-11-11 04:22:45.084000+00:00   
4         NaN           Area A_Wet Bulb 2018-11-11 04:22:45.084000+00:00   
5         NaN   Area A_Compressor Power 2018-11-10 16:31:09.311000+00:00   
6         NaN        Area C_Temperature 2018-11-10 16:31:09.311000+00:00   
7      Area A               Temperature 2019-09-07 13:23:27.130000+00:00   
8   Reactor 1    Solution Concentration 2025-04-27 16:39:22.442000+00:00   
9   Reactor 1                 Operation 2025-04-27 16:39:22.442000+00:00   
10  Reactor 1                  Batch ID 2025-04-27 16:39:22.442000+00:00   

                                End  \
0  2025-12-30 03:21:12.894076+00:00   
1  2025-12-30 01:21:12.894076+00:00   
2  2018-11-12 04:22:45.084000+00:00   
3  2025-12-30 01:21:12.894076+00:00   
4  2018-12-17 06:18:49.287000+00:00   
5  2025-12-30 01:21:12.894076+00:00   
6  2025-12-30 01:21:12.894076+00:00   
7  2019-10-07 23:23:27.130000+00:00   
8  2025-04-28 16:39:22.442000+00:00   
9  2025-04-28 16:39:22.442000+00:00   
10 2025-04-28 16:39:22.442000+00:00   

                                          Calculation  
0   $within = condition(\n   capsule("2018-11-01T1...  
1   $within = condition(\n   capsule("2018-11-01T1...  
2   $within = condition(\n   capsule("2018-11-01T1...  
3   $within = condition(\n   capsule("2018-11-11T0...  
4   $within = condition(\n   capsule("2018-11-11T0...  
5   $within = condition(\n   capsule("2018-11-10T1...  
6   $within = condition(\n   capsule("2018-11-10T1...  
7   $within = condition(\n   capsule("2019-09-07T1...  
8   $within = condition(\n   capsule("2025-04-27T1...  
9   $within = condition(\n   capsule("2025-04-27T1...  
10  $within = condition(\n   capsule("2025-04-27T1...

You can see `Start` and `End` columns that provide the overall time bounds that were detected in the workbook data references. There is also a `Calculation` column that refines this broad time period into specific "chunks" of data, defined as individual capsules and using the `within()` and `touches()` Seeq Formula functions to pull data only for those time periods -- not just everything between `Start` and `End`.

This manifest DataFrame can be fed directly into `spy.pull()` but it is recommended that you use `spy.workbooks.job.data.pull()` like so:

In [17]:
spy.workbooks.job.data.pull(job_folder)

ID  \
ID                                                                           
0F0E5194-F0F4-77F0-9282-75B6D1EDF8D0  0F0E5194-F0F4-77F0-9282-75B6D1EDF8D0   
0F0E5194-F480-FB30-9424-1E25FAFC6D1D  0F0E5194-F480-FB30-9424-1E25FAFC6D1D   
0F0E5194-F13D-FBD0-A828-C62D76DBB3C7  0F0E5194-F13D-FBD0-A828-C62D76DBB3C7   
0F0E5194-F120-7710-A8EE-4C2619321D62  0F0E5194-F120-7710-A8EE-4C2619321D62   
0F0E5194-F638-6270-B8CD-969173BDFC9F  0F0E5194-F638-6270-B8CD-969173BDFC9F   
0F0E5194-F646-ECD0-B161-3B58818B56EA  0F0E5194-F646-ECD0-B161-3B58818B56EA   
0F0E5194-F5EC-7780-835D-E33F795E19DB  0F0E5194-F5EC-7780-835D-E33F795E19DB   
0F0E5194-F02C-64D0-B9EC-35732C8E310E  0F0E5194-F02C-64D0-B9EC-35732C8E310E   
0F0E5194-F58D-6410-8DD8-F446DC388736  0F0E5194-F58D-6410-8DD8-F446DC388736   
0F0E5194-F5D4-60E0-A95A-86D0659E009B  0F0E5194-F5D4-60E0-A95A-86D0659E009B   
0F0E5194-F0D2-7510-8A50-ECD4A148A666  0F0E5194-F0D2-7510-8A50-ECD4A148A666   

                                              Type  \
ID                                                   
0F0E5194-F0F4-77F0-9282-75B6D1EDF8D0  StoredSignal   
0F0E5194-F480-FB30-9424-1E25FAFC6D1D  StoredSignal   
0F0E5194-F13D-FBD0-A828-C62D76DBB3C7  StoredSignal   
0F0E5194-F120-7710-A8EE-4C2619321D62  StoredSignal   
0F0E5194-F638-6270-B8CD-969173BDFC9F  StoredSignal   
0F0E5194-F646-ECD0-B161-3B58818B56EA  StoredSignal   
0F0E5194-F5EC-7780-835D-E33F795E19DB  StoredSignal   
0F0E5194-F02C-64D0-B9EC-35732C8E310E  StoredSignal   
0F0E5194-F58D-6410-8DD8-F446DC388736  StoredSignal   
0F0E5194-F5D4-60E0-A95A-86D0659E009B  StoredSignal   
0F0E5194-F0D2-7510-8A50-ECD4A148A666  StoredSignal   

                                                            Path      Asset  \
ID                                                                            
0F0E5194-F0F4-77F0-9282-75B6D1EDF8D0                         NaN        NaN   
0F0E5194-F480-FB30-9424-1E25FAFC6D1D                         NaN        NaN   
0F0E5194-F13D-FBD0-A828-C62D76DBB3C7                         NaN        NaN   
0F0E5194-F120-7710-A8EE-4C2619321D62                         NaN        NaN   
0F0E5194-F638-6270-B8CD-969173BDFC9F                         NaN        NaN   
0F0E5194-F646-ECD0-B161-3B58818B56EA                         NaN        NaN   
0F0E5194-F5EC-7780-835D-E33F795E19DB                         NaN        NaN   
0F0E5194-F02C-64D0-B9EC-35732C8E310E  Example >> Cooling Tower 1     Area A   
0F0E5194-F58D-6410-8DD8-F446DC388736       Example >> Batch Data  Reactor 1   
0F0E5194-F5D4-60E0-A95A-86D0659E009B       Example >> Batch Data  Reactor 1   
0F0E5194-F0D2-7510-8A50-ECD4A148A666       Example >> Batch Data  Reactor 1   

                                                          Name  \
ID                                                               
0F0E5194-F0F4-77F0-9282-75B6D1EDF8D0        Area A_Temperature   
0F0E5194-F480-FB30-9424-1E25FAFC6D1D   Area A_Compressor Stage   
0F0E5194-F13D-FBD0-A828-C62D76DBB3C7  Area A_Relative Humidity   
0F0E5194-F120-7710-A8EE-4C2619321D62          Area A_Optimizer   
0F0E5194-F638-6270-B8CD-969173BDFC9F           Area A_Wet Bulb   
0F0E5194-F646-ECD0-B161-3B58818B56EA   Area A_Compressor Power   
0F0E5194-F5EC-7780-835D-E33F795E19DB        Area C_Temperature   
0F0E5194-F02C-64D0-B9EC-35732C8E310E               Temperature   
0F0E5194-F58D-6410-8DD8-F446DC388736    Solution Concentration   
0F0E5194-F5D4-60E0-A95A-86D0659E009B                 Operation   
0F0E5194-F0D2-7510-8A50-ECD4A148A666                  Batch ID   

                                                       Time  Count  Pages  \
ID                                                                          
0F0E5194-F0F4-77F0-9282-75B6D1EDF8D0 0 days 00:00:02.840068  13330      1   
0F0E5194-F480-FB30-9424-1E25FAFC6D1D 0 days 00:00:02.352852   6874      1   
0F0E5194-F13D-FBD0-A828-C62D76DBB3C7 0 days 00:00:00.239767   5766      1   
0F0E5194-F120-7710-A8EE-4C2619321D62 0 days 00:00:02.330046    756      1   
0F0E5

Data has now been added to the job folder for the time periods identified in the manifest and can be pushed to the dummy items like so:

In [18]:
spy.workbooks.job.data.push(job_folder)

ID  \
0F0E5194-F120-7710-A8EE-4C2619321D62  0F0E51DD-89E7-7140-8E0B-D830C8445157   

                                              Type              Name  Count  \
0F0E5194-F120-7710-A8EE-4C2619321D62  StoredSignal  Area A_Optimizer    753   

                                      Pages                   Time   Result  
0F0E5194-F120-7710-A8EE-4C2619321D62      1 0 days 00:00:00.039959  Success

If you refresh the page of the **Details Pane** within the **Example Analysis**, you'll now see data for **Area A_Optimizer**.

However, if you move the Display Range to the left, you'll see that **Area A_Optimizer** data only exists for the time period that was originally on the screen.

If you want to expand how much data is pulled for a particular item, you can execute a command to alter the manifest and then pull/push again:

In [19]:
# Expand the time periods for Area A_Optimizer by 2 weeks on either side
spy.workbooks.job.data.expand(job_folder, {'Name': 'Area A_Optimizer'}, by='2w')

spy.workbooks.job.data.pull(job_folder, resume=False)
spy.workbooks.job.data.push(job_folder, resume=False)

ID  \
0F0E5194-F120-7710-A8EE-4C2619321D62  0F0E51DD-89E7-7140-8E0B-D830C8445157   

                                              Type              Name  Count  \
0F0E5194-F120-7710-A8EE-4C2619321D62  StoredSignal  Area A_Optimizer  30993   

                                      Pages                   Time   Result  
0F0E5194-F120-7710-A8EE-4C2619321D62      1 0 days 00:00:00.208054  Success

There are a series of functions to alter the manifest, and

- `spy.workbooks.job.data.expand()` - Expand the existing time periods.
- `spy.workbooks.job.data.add()` - Add a specific time period.
- `spy.workbooks.job.data.remove()` - Remove a specific time period.
- `spy.workbooks.job.data.calculation()` - Apply a specific calculation, such as resample().

Documentation for these functions is found under the **Detailed Help** section below.

## Redoing Specific Workbooks/Items

In the process of pushing and pulling, you will usually use the `errors='catalog'` flag, which means that errors will be enumerated but the operation will keep going if at all possible. When you resume an operation, those items that had errors will not be re-attempted, because SPy (by default) assumes that you don't care about them.

But you will often care about errors, and you will figure out how to fix them (say, by altering a Datasource Map). You can force SPy to redo a push or pull operation for a particular item or set of items using the `redo` family of functions:

In [20]:
spy.workbooks.job.redo(job_folder, example_analysis_workbook_id)

In [21]:
spy.workbooks.job.data.redo(job_folder, area_a_optimizer_id)

## Zip/Unzip the Job Folder

If you are intending to transfer workbook information to another Seeq server, it is convenient to package up the job folder as a zip file. There are two functions for this purpose:

In [22]:
spy.workbooks.job.zip(job_folder, overwrite=True)

In [23]:
spy.workbooks.job.unzip(job_folder + '.zip', overwrite=True)

## Detailed Help

All SPy functions have detailed documentation to help you use them. Just execute `help(spy.<func>)` like
you see below.

**Make sure you re-execute the cells below to see the latest documentation. It otherwise might be from an
earlier version of SPy.**

In [24]:
help(spy.workbooks.job.pull)

Help on function pull in module seeq.spy.workbooks.job._pull:

pull(
    job_folder: 'str',
    workbooks_df: 'Union[pd.DataFrame, str]',
    *,
    resume: 'bool' = True,
    include_referenced_workbooks: 'bool' = True,
    include_rendered_content: 'bool' = False,
    errors: 'Optional[str]' = None,
    quiet: 'Optional[bool]' = None,
    status: 'Optional[Status]' = None,
    session: 'Optional[Session]' = None
) -> 'pd.DataFrame'
    Pulls the definitions for each workbook specified by workbooks_df on to
    disk, in a restartable "job"-like fashion.

    Parameters
    ----------
    job_folder : {str}
        A full or partial path to a folder on disk where the workbooks
        definitions will be saved. If the folder does not exist, it will be
        created. If the folder exists, the job will continue where it left off.

    workbooks_df : {str, pd.DataFrame}
        A DataFrame containing 'ID', 'Type' and 'Workbook Type' columns that
        can be used to identify the workb

In [25]:
help(spy.workbooks.job.push)

Help on function push in module seeq.spy.workbooks.job._push:

push(
    job_folder,
    *,
    resume: 'bool' = True,
    path: 'str' = None,
    owner: 'str' = None,
    label: 'str' = None,
    datasource=None,
    use_full_path: 'bool' = False,
    access_control: 'str' = None,
    override_max_interp: 'bool' = False,
    global_inventory: 'Optional[str]' = None,
    create_dummy_items: 'bool' = False,
    dry_run: 'bool' = False,
    verbose: 'bool' = False,
    errors: 'Optional[str]' = None,
    quiet: 'Optional[bool]' = None,
    status: 'Optional[Status]' = None,
    session: 'Optional[Session]' = None,
    scope_globals_to_workbook: 'Optional[bool]' = None
) -> 'pd.DataFrame'
    Pushes the definitions for each workbook that was pulled by the
    spy.workbooks.job.pull() function, in a restartable "job"-like fashion.

    Parameters
    ----------
    job_folder : {str}
        A full or partial path to the job folder created by
        spy.workbooks.job.pull().

    resume :

In [26]:
help(spy.workbooks.job.data.pull)

Help on function pull in module seeq.spy.workbooks.job.data._pull:

pull(
    job_folder,
    *,
    resume: 'bool' = True,
    errors: 'Optional[str]' = None,
    quiet: 'Optional[bool]' = None,
    status: 'Optional[Status]' = None,
    session: 'Optional[Session]' = None
) -> 'pd.DataFrame'
    Pulls all the data that is used by the workbooks according to the Data
    Usages sections of the data_usage.json file in the job folder.

    Parameters
    ----------
    job_folder : {str}
        A full or partial path to the job folder created by
        spy.workbooks.job.pull().

    resume : bool, default True
        True if the pull should resume from where it left off, False if it
        should pull everything again.

    errors : {'raise', 'catalog'}, default 'raise'
        If 'raise', any errors encountered will cause an exception. If
        'catalog', errors will be added to a 'Result' column in the status.df
        DataFrame (errors='catalog' must be combined with
        st

In [27]:
help(spy.workbooks.job.data.manifest)
help(spy.workbooks.job.data.expand)
help(spy.workbooks.job.data.add)
help(spy.workbooks.job.data.remove)
help(spy.workbooks.job.data.calculation)

Help on function manifest in module seeq.spy.workbooks.job.data._pull:

manifest(job_folder, *, reset=False)
    Generates and returns a DataFrame with the list of items and data to be
    pulled. The manifest is initially generated by spy.workbooks.job.pull(),
    and is constructed by examining all of the Analyses and Topics that "touch"
    a signal or condition and noting the display ranges at play.

    You can modify the manifest using its sibling functions: expand(), add()
    or remove().

    The DataFrame returned is in a format suitable for spy.pull(), but in
    general it is expected that you will just use
    spy.workbooks.job.data.pull(), which has all the resume-ability of the
    spy.workbooks.job family of functions.

    Parameters
    ----------
    job_folder : {str}
        A full or partial path to the job folder created by
        spy.workbooks.job.pull().

    reset: {bool}, default False
        If True, the manifest will be reset to the original state (i.e. a

In [28]:
help(spy.workbooks.job.data.push)

Help on function push in module seeq.spy.workbooks.job.data._push:

push(
    job_folder,
    *,
    resume: 'bool' = True,
    replace: 'Optional[dict]' = None,
    datasource: 'str' = None,
    errors: 'Optional[str]' = None,
    quiet: 'Optional[bool]' = None,
    status: 'Optional[Status]' = None,
    session: 'Optional[Session]' = None
) -> 'pd.DataFrame'
    Pulls all the data that is used by the workbooks according to the Data
    Usages sections of the data_usage.json file in the job folder.

    Parameters
    ----------
    job_folder : {str}
        A full or partial path to the job folder created by
        spy.workbooks.job.pull() and populated with data by
        spy.workbooks.job.pull_data().

    resume : bool, default True
        True if the pull should resume from where it left off, False if it
        should pull everything again.

    replace : dict, default None
        A dict with the keys 'Start' and 'End'. If provided, any existing samples
        or capsules 

In [29]:
help(spy.workbooks.job.redo)
help(spy.workbooks.job.data.redo)

Help on function redo in module seeq.spy.workbooks.job._redo:

redo(
    job_folder: 'str',
    workbooks_df: 'Union[pd.DataFrame, str, list]' = None,
    *,
    action: 'Optional[str]' = None,
    hard: 'bool' = False,
    quiet: 'Optional[bool]' = None,
    status: 'Optional[Status]' = None
)
    Marks a set of workbooks to be redone in the specified job folder.

    Parameters
    ----------
    job_folder : {str}
        A full or partial path to the job folder.

    workbooks_df : {pd.DataFrame, str, list}
        A DataFrame containing an 'ID' column that can be used to identify the
        workbooks to affect. These IDs are based on the source system (not the
        destination system). Alternatively, you can supply a workbook ID
        directly as a str or list of strs. If omitted, all workbooks in the
        job folder are affected.

    action : str
        If supplied, limits the redo to the specified actions. You can specify
        'pull' or 'push'. If not supplied, bot